In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import risktools as rt
from loader import _load_json
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
up = {"m*": {"user": "", "pass": ""}, "eia": "", "quandl": ""}

up["eia"] = os.getenv("EIA")
up["quandl"] = os.getenv("QUANDL")
up["m*"]["pass"] = os.getenv("MS_PASS")
up["m*"]["user"] = os.getenv("MS_USER")


In [24]:
rt.data.get_names()

['cancrudeassays',
 'cancrudeassayssum',
 'cancrudeprices',
 'crudeassaysBP',
 'crudeassaysXOM',
 'crudes',
 'dflong',
 'dfwide',
 'df_fut',
 'eiaStocks',
 'eiaStorageCap',
 'expiry_table',
 'fizdiffs',
 'holidaysOil',
 'planets',
 'ref_opt_inputs',
 'ref_opt_outputs',
 'tickers_eia',
 'tradeCycle',
 'tradeprocess',
 'usSwapCurves',
 'usSwapCurvesPar',
 'usSwapIR',
 'usSwapIRdef']

In [26]:
for nm in rt.data.get_names():
    display(rt.data.open_data(nm))

,Ticker,Crude,date,Location,Density,Gravity,Sulphur,MCR,Nickel,Vanadium,...,Decanes,Benzene,Toluene,Ethyl Benzene,Xylenes,Sediment,TAN,Salt,Viscosity,Olefins
0,AWB,Access Western Blend,1970-01-01 00:00:00.000014610,Edmonton,919.9,22.2,3.83,10.3,68.7,184.5,...,0.56,0.28,0.58,0.07,0.50,72.0,1.62,7.5,NaN,NaN
1,AWB,Access Western Blend,1970-01-01 00:00:00.000014641,Edmonton,918.1,22.5,3.89,10.5,68.3,186.6,...,0.60,0.32,0.63,0.08,0.54,NaN,1.60,NaN,NaN,NaN
2,AWB,Access Western Blend,1970-01-01 00:00:00.000014669,Edmonton,917.9,22.5,3.79,10.3,66.3,178.2,...,0.58,0.32,0.60,0.07,0.52,NaN,1.63,NaN,NaN,NaN
3,AWB,Access Western Blend,1970-01-01 00:00:00.000014700,Edmonton,920.6,22.1,3.82,10.4,70.1,184.0,...,0.73,0.30,0.58,0.08,0.53,120.0,1.59,5.7,NaN,NaN
4,AWB,Access Western Blend,1970-01-01 00:00:00.000014730,Edmonton,924.2,21.5,3.90,10.5,69.3,190.9,...,0.63,0.29,0.56,0.07,0.51,NaN,1.70,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,WH,Wabasca Heavy,1970-01-01 00:00:00.000018901,Edmonton,935.7,19.6,3.92,8.6,49.3,120.0,...,1.81,0.05,0.14,0.12,0.36,NaN,0.83,NaN,NaN,NaN
1220,WH,Wabasca Heavy,1970-01-01 00:00:00.000018932,Edmonton,926.8,21.0,3.92,8.2,47.2,124.0,...,1.68,0.05,0.15,0.11,0.32,NaN,0.87,NaN,NaN,NaN
1221,WH,Wabasca Heavy,1970-01-01 00:00:00.000018962,Edmonton,921.7,21.9,3.97,8.0,53.0,167.0,...,2.06,0.09,0.28,0.15,0.51,98.0,0.78,5.3,NaN,NaN
1222,WH,Wabasca Heavy,1970-01-01 00:00:00.000018993,Edmonton,923.5,21.6,4.04,8.2,47.0,128.0,...,1.76,0.07,0.22,0.13,0.44,NaN,0.80,NaN,NaN,NaN


,Ticker,Crude,date,Density,Gravity,Sulphur,MCR,Nickel,Vanadium,C3-,...,Hexanes,Heptanes,Octanes,Nonanes,Decanes,Benzene,Toluene,Ethyl Benzene,Xylenes,TAN
0,AWB,Access Western Blend,1970-01-01 00:00:00.000016795,922.4403,21.7755,3.8900,10.6441,69.9540,182.8655,0.0668,...,6.3045,3.9330,2.4098,1.2527,0.5888,0.2351,0.4117,0.0450,0.3453,1.7029
1,BRN,Bow River North,1970-01-01 00:00:00.000016867,925.0811,21.3370,2.9093,10.1401,45.7906,103.0157,0.1400,...,4.1692,3.1091,2.7046,2.0180,0.9502,0.1086,0.2272,0.0769,0.2877,0.8817
2,CDB,Christina Dilbit Blend,1970-01-01 00:00:00.000017171,923.0045,21.6793,3.8504,10.4042,67.8603,175.1818,0.0621,...,5.9238,3.5237,2.1433,1.1469,0.5588,0.2049,0.3364,0.0374,0.2839,1.5869
3,CL,Cold Lake,1970-01-01 00:00:00.000016795,926.2164,21.1497,3.7587,10.3723,62.5993,160.0605,0.0588,...,5.3240,3.3108,2.1845,1.3508,0.6687,0.1981,0.3385,0.0451,0.3025,1.0840
4,LLB,Lloyd Blend,1970-01-01 00:00:00.000016736,927.1219,21.0000,3.5063,9.6292,55.6905,123.3672,0.0757,...,3.9163,2.9043,2.1555,1.5222,0.7750,0.1472,0.3001,0.0464,0.2945,0.8616
5,LLK,Lloyd Kerrobert,1970-01-01 00:00:00.000016785,928.6935,20.7377,3.1279,9.9006,46.6399,106.4920,0.0884,...,3.7816,2.4503,1.8656,1.4348,0.7070,0.1217,0.2051,0.0494,0.2131,0.9983
6,MSW,Mixed Sweet Blend,1970-01-01 00:00:00.000016420,823.7091,40.1473,0.4038,1.8118,4.1989,8.2978,0.5291,...,5.9105,7.2881,7.2419,5.5352,2.5027,0.2565,0.7993,0.2351,1.0463,0.0000
7,WCS,Western Canadian Select,1970-01-01 00:00:00.000016537,928.1290,20.8369,3.5406,9.7279,58.1678,138.0822,0.0744,...,3.9517,2.8157,2.1353,1.5397,0.7648,0.1415,0.2707,0.0518,0.2779,0.9762
8,WH,Wabasca Heavy,1970-01-01 00:00:00.000016734,932.1056,20.1833,4.1021,8.6681,52.3206,141.1627,0.0904,...,2.7012,2.8669,3.1208,2.7387,1.3591,0.0860,0.2597,0.1241,0.4548,1.0079


,Ticker,date,Value
0,AHS,2018-11-01,-48.5695
1,AHS,2018-10-01,-31.8708
2,AHS,2018-09-01,-32.6978
3,AHS,2018-08-01,-21.8058
4,AHS,2018-07-01,-19.8540
...,...,...,...
3026,WH,2001-05-01,-11.9757
3027,WH,2001-04-01,-11.6504
3028,WH,2001-03-01,-12.2358
3029,WH,2001-02-01,-13.4429


specifications               Start (C API) End (C API)  \
crude         cut                                        
HLS           AtRes1                   369          NA   
              AtRes2                   509          NA   
              Heavy.Naphtha1            95         149   
              Heavy.Naphtha2           149         175   
              Heavy.VGO1               509         550   
...                                    ...         ...   
thunder-horse Light.Naphtha             NA          95   
              Light.VGO                369         509   
              VacRes1                  550          NA   
              VacRes2                  585          NA   
              Whole.crude               NA          NA   

specifications               Yield on crude (perc wt)  \
crude         cut                                       
HLS           AtRes1                            41.35   
              AtRes2                             20.4   
              Heavy.Naphtha1                    10.15   
              Heavy.Naphtha2                      4.5   
              Heavy.VGO1                         4.85   
...                                               ...   
thunder-horse Light.Naphtha                      7.15   
              Light.VGO                          21.4   
              VacRes1                           18.75   
              VacRes2                            15.1   
              Whole.crude                         100   

specifications               Yield on crude (perc vol)  \
crude         cut                                        
HLS           AtRes1                             36.55   
              AtRes2                              17.2   
              Heavy.Naphtha1                      11.4   
              Heavy.Naphtha2                       4.9   
              Heavy.VGO1                           4.3   
...                                                ...   
thunder-horse Light.Naphtha                          9   
              Light.VGO                           19.9   
              VacRes1                            15.55   
              VacRes2                             12.3   
              Whole.crude                          100   

specifications               Density at 15C (kg/litre)  \
crude         cut                                        
HLS           AtRes1                            0.9551   
              AtRes2                            0.9987   
              Heavy.Naphtha1                    0.7515   
              Heavy.Naphtha2                    0.7722   
              Heavy.VGO1                        0.9461   
...                                                ...   
thunder-horse Light.Naphtha                     0.6765   
              Light.VGO                         0.9181   
              VacRes1                           1.0316   
              VacRes2                           1.0474   
              Whole.crude                       0.8572   

specifications               Total Sulphur (perc wt)  \
crude         cut                                      
HLS           AtRes1                           0.979   
              AtRes2                            1.36   
              Heavy.Naphtha1                   0.007   
              Heavy.Naphtha2                   0.019   
              Heavy.VGO1                       0.885   
...                                              ...   
thunder-horse Light.Naphtha                       NA   
              Light.VGO                        0.956   
              VacRes1                           2.23   
              VacRes2                            2.4   
              Whole.crude                      0.812   

specifications               Mercaptan Sulphur (ppm wt)  \
crude         cut                                         
HLS           AtRes1                                 NA   
              AtRes2                                 NA   
              Heavy.Naphtha1

Error appending Liza


...

,date,CL22H,HO22F,HO22H,LT22H
0,2019-01-02,51.93,1.8420,1.8433,-0.0450
1,2019-01-03,52.35,1.8616,1.8629,-0.0450
2,2019-01-04,52.79,1.8705,1.8718,-0.0450
3,2019-01-07,52.80,1.8759,1.8772,-0.0450
4,2019-01-08,53.85,1.9150,1.9163,-0.0450
...,...,...,...,...,...
752,2021-12-27,75.18,2.3535,2.3259,-0.0535
753,2021-12-28,75.60,2.3714,2.3421,-0.0535
754,2021-12-29,76.18,2.3778,2.3488,-0.0535
755,2021-12-30,76.61,2.3959,2.3675,-0.0535


{'times': array([ 0.        ,  0.08333333,  0.16666667,  0.25      ,  0.33333333,
         0.41666667,  0.5       ,  0.58333333,  0.66666667,  0.75      ,
         0.83333333,  0.91666667,  1.        ,  1.08333333,  1.16666667,
         1.25      ,  1.33333333,  1.41666667,  1.5       ,  1.58333333,
         1.66666667,  1.75      ,  1.83333333,  1.91666667,  2.        ,
         2.08333333,  2.16666667,  2.25      ,  2.33333333,  2.41666667,
         2.5       ,  2.58333333,  2.66666667,  2.75      ,  2.83333333,
         2.91666667,  3.        ,  3.08333333,  3.16666667,  3.25      ,
         3.33333333,  3.41666667,  3.5       ,  3.58333333,  3.66666667,
         3.75      ,  3.83333333,  3.91666667,  4.        ,  4.08333333,
         4.16666667,  4.25      ,  4.33333333,  4.41666667,  4.5       ,
         4.58333333,  4.66666667,  4.75      ,  4.83333333,  4.91666667,
         5.        ,  5.08333333,  5.16666667,  5.25      ,  5.33333333,
         5.41666667,  5.5       ,  5.58333

{'times': array([ 0.        ,  0.08333333,  0.16666667,  0.25      ,  0.33333333,
         0.41666667,  0.5       ,  0.58333333,  0.66666667,  0.75      ,
         0.83333333,  0.91666667,  1.        ,  1.08333333,  1.16666667,
         1.25      ,  1.33333333,  1.41666667,  1.5       ,  1.58333333,
         1.66666667,  1.75      ,  1.83333333,  1.91666667,  2.        ,
         2.08333333,  2.16666667,  2.25      ,  2.33333333,  2.41666667,
         2.5       ,  2.58333333,  2.66666667,  2.75      ,  2.83333333,
         2.91666667,  3.        ,  3.08333333,  3.16666667,  3.25      ,
         3.33333333,  3.41666667,  3.5       ,  3.58333333,  3.66666667,
         3.75      ,  3.83333333,  3.91666667,  4.        ,  4.08333333,
         4.16666667,  4.25      ,  4.33333333,  4.41666667,  4.5       ,
         4.58333333,  4.66666667,  4.75      ,  4.83333333,  4.91666667,
         5.        ,  5.08333333,  5.16666667,  5.25      ,  5.33333333,
         5.41666667,  5.5       ,  5.58333

,d1d,d1w,d1m,d3m,d6m,d1y,s2y,s3y,s5y,s7y,...,s20y,s30y,fut1,fut2,fut3,fut4,fut5,fut6,fut7,fut8
date,,,,,,,,,,,,,,,,,,,,,
2021-05-10,0.0006,0.0007,0.0010,0.0017,0.0019,0.0027,0.0025,0.0041,0.0085,0.0121,...,0.0194,0.0199,99.8300,99.820,99.775,99.800,99.770,99.720,99.625,99.560
2021-05-11,0.0006,0.0007,0.0009,0.0016,0.0019,0.0027,0.0025,0.0043,0.0088,0.0125,...,0.0200,0.0206,99.8300,99.820,99.775,99.795,99.760,99.705,99.610,99.540
2021-05-12,0.0006,0.0007,0.0010,0.0015,0.0019,0.0026,0.0027,0.0047,0.0094,0.0131,...,0.0204,0.0209,99.8350,99.825,99.775,99.790,99.745,99.680,99.565,99.485
2021-05-13,0.0006,0.0007,0.0010,0.0016,0.0019,0.0026,0.0026,0.0045,0.0092,0.0129,...,0.0202,0.0208,99.8400,99.830,99.785,99.800,99.760,99.695,99.585,99.510
2021-05-14,0.0006,0.0007,0.0010,0.0016,0.0019,0.0027,0.0026,0.0045,0.0090,0.0126,...,0.0198,0.0204,99.8400,99.830,99.785,99.800,99.760,99.695,99.585,99.510
2021-05-17,0.0006,0.0007,0.0010,0.0015,0.0019,0.0026,0.0025,0.0045,0.0091,0.0127,...,0.0198,0.0202,99.8475,99.840,99.795,99.810,99.765,99.700,99.590,99.515
2021-05-18,0.0006,0.0007,0.0010,0.0016,0.0018,0.0026,0.0025,0.0044,0.0091,0.0127,...,0.0200,0.0205,99.8500,99.845,99.800,99.815,99.770,99.705,99.595,99.520
2021-05-19,0.0006,0.0007,0.0010,0.0015,0.0018,0.0026,0.0025,0.0044,0.0090,0.0126,...,0.0198,0.0203,99.8500,99.845,99.795,99.810,99.765,99.695,99.575,99.495
2021-05-20,0.0006,0.0006,0.0009,0.0015,0.0018,0.0026,0.0025,0.0045,0.0091,0.0127,...,0.0199,0.0204,99.8550,99.850,99.800,99.815,99.775,99.710,99.600,99.525


,tickQL,type,source,tickSource
0,d1d,ICE_LIBOR,FRED,USDONTD156N
1,d1w,ICE_LIBOR,FRED,USD1WKD156N
2,d1m,ICE_LIBOR,FRED,USD1MTD156N
3,d3m,ICE_LIBOR,FRED,USD3MTD156N
4,d6m,ICE_LIBOR,FRED,USD6MTD156N
5,d1y,ICE_LIBOR,FRED,USD12MD156N
6,fut1,EuroDollar,quandl,CHRIS/CME_ED1
7,fut2,EuroDollar,quandl,CHRIS/CME_ED2
8,fut3,EuroDollar,quandl,CHRIS/CME_ED3
9,fut4,EuroDollar,quandl,CHRIS/CME_ED4


In [ ]:
rt.simOUJ(theta=5,dt=1/252, mr_lag=15, T=5).iloc[:,:100].plot(legend=False, figsize=(20,10))

In [ ]:
mvgbm = rt.MVGBM(
    s0=[100,100,100], 
    r=0.0, 
    sigma=[.2,.2,.2], 
    T=0.25, 
    dt=1/252, 
    cor=[[1,0.5,0.5],[0.5,1,0.5],[0.5,0.5,1]],
    asset_names=['A','B','C']
    )
mvgbm.fit()
mvgbm.simulate()
mvgbm.plot_efficient_frontier(strike=100)

In [ ]:
i = np.random.randint(0,1000)
print(i)
prices = pd.DataFrame(mvgbm.sims[:,i,:])
prices

In [ ]:
returns = np.log(prices) - np.log(prices).shift(1)

In [ ]:
returns.dropna().std()*np.sqrt(252)

In [ ]:
mvgbm2 = rt.MVGBM(
    r=0.0, 
    T=0.25, 
    dt=1/252, 
    prices=prices
    )
mvgbm2.fit()
mvgbm2._s0=[100,100,100]
mvgbm2.simulate()
mvgbm2.plot_efficient_frontier(strike=100, portfolio_sims=10000)

In [ ]:
pd.DataFrame(mvgbm2.sims[:,:,0]).iloc[:,:100].plot(legend=False)

In [ ]:
mvgbm2._sigma

In [ ]:
mvgbm2._cor